## Page Content & Link & Description Dataframe (Beautifulsoup)

<br>

### Development Environment

In [1]:
import re
import requests
import urllib
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlretrieve, urlopen
from urllib.error import URLError, HTTPError

### Page Information

In [2]:
def make_page_content_link_description_list(soup):

    html_tags = ['div', 'img', 'description', 'thumbnail']
    title_list = []
    link_list = []
    description_list = []

    a = soup.select("a")
    for a_text in a:
        a_text = str(a_text)

        if 'detail' in a_text:
            a_title = re.search('/">(.+?)</a>', a_text)
            if a_title != None:
                a_title = a_title.group()
                if all(html_tag not in a_title for html_tag in html_tags):
                    a_title = a_title.replace('/">', '')
                    a_title = a_title.replace('</a>', '')
                    title_list.append(a_title)

            a_link = re.search('<a href=(.+?)/">', a_text)
            if a_link != None:
                a_link = a_link.group()
                a_link = a_link.replace('<a href="', 'https://yozm.wishket.com')
                a_link = a_link.replace('">', '')
                link_list.append(a_link)
                link_list = dict.fromkeys(link_list)
                link_list = list(link_list)

        a_description = re.search('<p class="item-description">(.+?)</p>', a_text)
        if a_description != None:
            a_description = a_description.group()
            a_description = a_description.replace('<p class="item-description">', '')
            a_description = a_description.replace('</p>', '')
            description_list.append(a_description)

    return title_list, link_list, description_list

In [58]:
def make_total_page_content_link_description_list(link_pattern, page_topic, number_of_page):

    total_title_list = []
    total_link_list = []
    total_description_list = []
    total_topic_list = []

    # query = urllib.parse.quote('keyword')
    for page_num in range(number_of_page):
        
        url = link_pattern[0] + page_topic + link_pattern[1] + str(page_num + 1) + link_pattern[2] # + query

        try:
            request = urllib.request.Request(url)

            response = urllib.request.urlopen(request)
            response_code = response.getcode()

            if response_code == 200:
                response_body = response.read()

            soup = BeautifulSoup(response_body, 'html.parser') 

            title_list, link_list, description_list = make_page_content_link_description_list(soup)

            total_title_list.append(title_list)
            total_link_list.append(link_list)
            total_description_list.append(description_list)

        except HTTPError as e:
            err = e.read()
            code = e.getcode()
            print("Topic:", page_topic[0].upper() + page_topic[1:])
            print("The number of page:", page_num - 1) 
            print("Error Code:", code)
            print("\n")
            break

    total_title_list = sum(total_title_list, [])
    total_link_list = sum(total_link_list, [])
    total_description_list = sum(total_description_list, [])
    total_topic_list = [page_topic] * len(total_title_list)

    return total_topic_list, total_title_list, total_link_list, total_description_list

### Make Page Content & Link & Description Dataframe

In [46]:
def make_page_content_link_description_df(topics, titles, links, descriptions):
    
    page_content_link_description_df = pd.DataFrame({'topic':topics, 'title':titles, 'link':links, 'description':descriptions,})
    
    return page_content_link_description_df

In [59]:
link_pattern = ['https://yozm.wishket.com/magazine/list/', '/?page=', '&sort=&q=']
page_topics = ['plan', 'design', 'product', 'outsourcing', 'freelancing', 'business', 'develop']
number_of_page = 1000

page_content_link_description_df = pd.DataFrame({'topic':["topic"], 'title':["title"], 'link':["link"], 'description':["description"],})

for page_topic in page_topics:
    topic_list, title_list, link_list, description_list = make_total_page_content_link_description_list(link_pattern, page_topic, number_of_page)
    part_page_content_link_description_df = make_page_content_link_description_df(topic_list, title_list, link_list, description_list)
    page_content_link_description_df = pd.concat([page_content_link_description_df, part_page_content_link_description_df])

page_content_link_description_df = page_content_link_description_df.reset_index()
del page_content_link_description_df['index']
page_content_link_description_df = page_content_link_description_df.drop(page_content_link_description_df.index[[0]])

Topic: Plan
The number of page: 63
Error Code: 404


Topic: Design
The number of page: 34
Error Code: 404


Topic: Product
The number of page: 34
Error Code: 404


Topic: Outsourcing
The number of page: 3
Error Code: 404


Topic: Freelancing
The number of page: 5
Error Code: 404


Topic: Business
The number of page: 9
Error Code: 404


Topic: Develop
The number of page: 58
Error Code: 404




In [80]:
pd.set_option('display.max_colwidth', None)
page_content_link_description_df

,topic,title,link,description
1,plan,"기획자가 좋은 피드백을 이끌어내는 방법, 피드포워드",https://yozm.wishket.com/magazine/detail/2181/,"기획 일의 대부분은 협업으로 이뤄지고 그 협업 역시 두 사람이 같은 작업을 동시에 하는 것이 아니라, 각자가 맡은 역할을 주고받으며 끊임없이 소통합니다. 그러니 협업을 잘한다는 것의 진짜 의미는 피드백을 잘 주고받는 것이라고도 할 수 있고, 이를 위해서는 좋은 피드백을 설계하는 '피드포워드(feedforward)' 능력이 절실히 필요합니다."
2,plan,"서비스 기획 실전편: 기능 명세서, 스토리보드 작성하기",https://yozm.wishket.com/magazine/detail/2169/,"지난 글에서는 시장을 이해하고 사업적으로 유효한 기능을 정의하는 과정에 대해 설명했습니다. 이번 글은 협업을 위해 이러한 내용들을 구체화시키는 과정을 다룹니다. 서비스 기획은 프로젝트의 규모에 따라 혼자 할 수도 있지만, 서비스를 만드는 것 자체는 혼자 할 수 없습니다. (대체로 그렇다는 의미입니다.) 성공적인 서비스를 론칭하기 위해 어떤 기능을 만들지 정리가 되었다면, 개발과 디자인에 필요한 문서들을 작성해야 합니다."
3,plan,효과적인 설문 조사를 위한 3단계,https://yozm.wishket.com/magazine/detail/2167/,"급격히 진화하는 고객의 니즈에 대응하기 위해서는 이에 걸맞은 발 빠른 서비스 개발 환경이 요구됩니다. 이를 위해 비교적 저비용으로 많은 양의 사용자 데이터를 효율적으로 수집할 수 있는 사용자 조사 방법이 필요한데요. 대표적인 방법으로 설문 조사를 들 수 있습니다. 이번 글에서는 가설적인 고객 니즈를 효율적으로 검증하기 위해, 설문 조사를 어떻게 기획하고 수행해야 하는지 알아보겠습니다."
4,plan,커머스 사례로 보는 인풋 지표와 아웃풋 지표,https://yozm.wishket.com/magazine/detail/2166/,지표에 관심을 갖고 자료를 공부해 보신 분이라면 인풋 지표(input metrics)와 아웃풋 지표(output metrics)라는 용어를 접해 보신 적이 있을 겁니다. 아웃풋 지표는 인풋 지표가 작용한 결과로 나타나는 지표입니다. 이번 글에서는 지표의 위계인 인풋 지표와 아웃풋 지표 개념을 소개해 드리겠습니다.
5,plan,구글 성공 만든 ‘도그푸딩(Dogfooding)’ 리서치 방법론 적용기,https://yozm.wishket.com/magazine/detail/2156/,"도그푸딩이라는 리서치 방법에 대해 들어보셨나요? 피엑스디에서도 프로젝트 중 도그푸딩을 몇 번 시도했었습니다. 아직 시장 성숙도가 낮은 블록체인 관련 프로덕트를 리딩하며 느낀 점은 사용자를 이해하는 것에 앞서 팀이 프로덕트 관련된 지식을 잘 쌓을 수 있게 돕는 것이 무엇보다 중요하다는 것이었습니다. 특히 간단한 설명만으로는 쉽게 이해한다는 수준에 도달하기 어려운 비즈니스이기에 스스로 만들고 있는 프로덕트가 어떤 환경에서 사용될지 상상하기 어려운 경우도 많은데요, 팀의 이해도를 높이기 위해 시도해 본 여러 방법 중에서 도그푸딩을 시도해 본 경험을 소개해 보려고 합니다."
...,...,...,...,...
2099,develop,"나한테 딱 맞는 홈페이지 만드는 방법, 어떻게 찾을 수 있을까요?",https://yozm.wishket.com/magazine/detail/36/,"홈페이지 제작을 고민하고 계신가요? 그렇다면 홈페이지 만드는 방법은 얼마나 다양한지, 장단점은 무엇이 있는지 알아봐야할텐데요. 이번 그린 클라이언트에서 홈페이지 만드는 방법을 총정리 했습니다. 나한테 딱 맞는 홈페이지 만드는 방법, 알아보세요:)"
2100,develop,"떠오르는 블루오션, 우주 인터넷 알아보기",https://yozm.wishket.com/magazine/detail/38/,떠오르는 블루오션 우주 인터넷. 유명 ICT기업들이 앞다투어 투자중인데요! 이번 그린 클라이언트에서는 우주 인터넷의 개념과 여러 기업들이 진행중인 우주 인터넷 사업에 대해 알려드립니다. 요즘 뜨고 있는 우주 인터넷 알아보고 가세요:)
2101,develop,"웹/ 앱 서비스 기획 전, 꼭 필요한 체크리스트",https://yozm.wishket.com/magazine/detail/40/,"웹이든 앱이든 서비스 기획 단계에서부터 막막함을 느끼시는 분들이 많을겁니다. 그렇다면 앱/ 웹 서비스 기획, 도대체 어떻게 해야 잘할 수 있는 것일까요? 오늘 그린 클라이언트에서는 앱/ 웹 서비스 기획 과정에서 반드시 확인해봐야할 여섯가지 체크리스트를 준비해봤습니다:)"
2102,develop,"다들 호스팅, 호스팅 하는데, 호스팅이란 무엇일까?",https://yozm.wishket.com/magazine/detail/43/,"홈페이지 제작을 준비할 때, '호스팅'이라는 말을 많이 듣게 됩니다. 하지만 호스팅이 정확히 무엇인지, 종류에는 어떤 게 있는지 헷갈릴 때가 있지요. 이번 그린 클라이언트는 그런 여러분들을 위해 호스팅에 대해 총정리 해보았습니다. 지금 바로 읽어보세요:)"


In [81]:
page_content_link_description_df.to_excel('page_content_link_description_df.xlsx')

<br>

### Reference

<b>Github</b>
<br>cheris8
<br>[동적 웹 페이지 크롤링 with Python](https://cheris8.github.io/data%20analysis/DC-Dynamic-Webpage-Crawling/)

<br><b>Magazine</b>
<br>[요즘IT](https://yozm.wishket.com/magazine/)